In [113]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

In [114]:
os.chdir('Data')
stats_data = pd.read_csv('Regular-Advanced.csv')
defense_data = pd.read_csv('Defensive.csv')
hustle_data = pd.read_csv('Hustle.csv')
shooting_data = pd.read_csv('Shooting.csv')
play_type_data = pd.read_csv('Play Type.csv')
passing_data = pd.read_csv('Passing.csv')

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'Data'

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
pca = PCA(.75, random_state = 0)

In [4]:
class PCA:

    def __init__(self, data):
        self.data = data
        self.PCA = pca.fit(self.Standard_Scaler_Preprocess().drop(['PLAYER', 'TEAM', 'POSITION'], axis = 1))
        
    def Standard_Scaler_Preprocess(self):    
        std_scale = StandardScaler()
        std_scale_data = std_scale.fit_transform(self.data.drop(['PLAYER', 'TEAM', 'POSITION'], axis = 1))
        std_scale_data = pd.DataFrame(std_scale_data, columns = self.data.drop(['PLAYER', 'TEAM', 'POSITION'], axis = 1).columns.tolist())
        std_scale_data['PLAYER'] = self.data['PLAYER']
        std_scale_data['TEAM'] = self.data['TEAM']
        std_scale_data['POSITION'] = self.data['POSITION']
        return std_scale_data
    
    def PCA_name(self):
        PCA_name = []
        for i in range(1, self.PCA.n_components_ + 1):
            PCA_name += ['PC' + str(i)]
        return PCA_name
    
    def PCA_variance(self):
        pca_variance = pd.DataFrame({"Variance Explained" : self.PCA.explained_variance_,
                                     'Percentage of Variance Explained' : self.PCA.explained_variance_ratio_}, index = self.PCA_name())
        pca_variance['Percentage of Variance Explained'] = (pca_variance['Percentage of Variance Explained'] * 100).round(0)
        pca_variance['Cumulative Percentage of Variance Explained'] = pca_variance['Percentage of Variance Explained'].cumsum()
        return pca_variance
    
    def PCA_transform(self):
        pca_data = pca.fit_transform(self.Standard_Scaler_Preprocess().drop(['PLAYER', 'TEAM', 'POSITION'], axis = 1))
        pca_data = pd.DataFrame(pca_data, columns = self.PCA_name())
        pca_data['PLAYER'] = self.Standard_Scaler_Preprocess()['PLAYER']
        pca_data['TEAM'] = self.Standard_Scaler_Preprocess()['TEAM']
        pca_data['POSITION'] = self.Standard_Scaler_Preprocess()['POSITION']
        return pca_data
    
    def Heatmap(self): 
        pca_eigen = pd.DataFrame(self.PCA.components_, columns = self.Standard_Scaler_Preprocess().drop(['PLAYER', 'TEAM', 'POSITION'], axis = 1).columns.tolist(), index = self.PCA_name()).T
        plt.figure(figsize = (10,10))
        sns.heatmap(pca_eigen.abs(), vmax = 0.5, vmin = 0)
        
    def PCA_sorted_eigen(self, PC):
        pca_eigen = pd.DataFrame(self.PCA.components_, columns = self.Standard_Scaler_Preprocess().drop(['PLAYER', 'TEAM', 'POSITION'], axis = 1).columns.tolist(), index = self.PCA_name()).T
        return pca_eigen.loc[pca_eigen[PC].abs().sort_values(ascending = False).index][PC]


In [5]:
def pca_name_change(df, name):
    for column in df.columns.tolist():
        if column[0:2] == 'PC':\
            df.rename({column : name + '_' + column}, axis = 1, inplace = True)
    return df

In [6]:
pca_df = pd.DataFrame()

In [7]:
all_data = stats_data.merge(defense_data)
all_data = all_data.merge(hustle_data)
all_data = all_data.merge(shooting_data)
all_data = all_data.merge(play_type_data)
all_data = all_data.merge(passing_data)
print(all_data.shape)
all_data.head()

(388, 86)


,PLAYER,TEAM,POSITION,PTS,FG%,3P%,OREB,DREB,AST,STL,BLK,+/-,AST%,AST/TO,OREB%,DREB%,TO RATIO,TS%,USG%,PIE,<6_FREQ,<6_DFGM,<6_DIFF%,6-9_FREQ,6-9_DFGM,6-9_DIFF%,10-15_FREQ,10-15_DFGM,10-15_DIFF%,16-3PT_FREQ,16-3PT_DFGM,16-3PT_DIFF%,3PT_FREQ,3PT_DFGM,3PT_DIFF%,SCREEN ASSISTS,DEFLECTIONS,LOOSE BALLS RECOVERED,CHARGES DRAWN,CONTESTED 2PT SHOTS,CONTESTED 3PT SHOTS,FGM_<5,FG%_<5,FGM_5-9,FG%_5-9,FGM_10-14,FG%_10-14,FGM_15-19,FG%_15-19,FGM_20-24,FG%_20-24,FGM_25-29,FG%_25-29,ISOLATION_FREQ,ISOLATION_FGM,ISOLATION_FG%,PNR_HANDLER_FREQ,PNR_HANDLER_FGM,PNR_HANDLER_FG%,PNR_ROLL_FREQ,PNR_ROLL_FGM,PNR_ROLL_FG%,TRANSITION_FREQ,TRANSITION_FGM,TRANSITION_FG%,POST_UP_FREQ,POST_UP_FGM,POST_UP_FG%,SPOT_UP_FREQ,SPOT_UP_FGM,SPOT_UP_FG%,HANDOFF_FREQ,HANDOFF_FGM,HANDOFF_FG%,CUT_FREQ,CUT_FGM,CUT_FG%,OFF_SCREEN_FREQ,OFF_SCREEN_FGM,OFF_SCREEN_FG%,PUTBACK_FREQ,PUTBACK_FGM,PUTBACK_FG%,PASSES MADE,SECONDARY AST,AST TO PASS% ADJ
0,James Harden,HOU,G,36.1,44.2,36.8,0.8,5.8,7.5,2.0,0.7,4.6,39.4,1.51,2.3,15.7,12.0,61.6,39.6,20.1,30.9,2.5,3.9,14.6,0.5,-24.9,8.1,0.5,3.7,9.8,0.4,-3.7,36.8,1.6,-0.9,0.3,3.5,1.5,0.08,4.4,2.4,4.2,56.2,1.2,49.2,0.4,46.5,0.2,36.4,0.9,34.4,3.9,37.7,0.487,7.9,0.401,0.221,2.7,0.477,0.000,0.0,0.000,0.133,1.3,0.534,0.009,0.1,0.647,0.032,0.6,0.449,0.036,0.4,0.478,0.014,0.1,0.682,0.009,0.2,0.316,0.017,0.1,0.591,49.5,0.9,17.8
1,Paul George,OKC,F,28.0,43.8,38.6,1.4,6.8,4.1,2.2,0.4,6.4,17.3,1.55,3.3,17.3,8.7,58.3,28.5,16.1,33.7,2.6,1.0,7.3,0.3,-18.5,6.5,0.3,-5.4,8.1,0.5,12.1,44.0,1.8,-1.7,0.5,3.8,2.1,0.06,4.3,4.2,3.0,59.0,0.5,37.9,0.7,40.7,0.9,39.3,1.7,42.1,2.3,37.2,0.134,1.9,0.368,0.245,2.7,0.449,0.014,0.3,0.241,0.206,2.3,0.482,0.036,0.3,0.556,0.117,1.5,0.465,0.054,0.7,0.404,0.019,0.1,0.720,0.098,1.3,0.419,0.000,0.0,0.000,31.1,0.3,15.0
2,Giannis Antetokounmpo,MIL,F,27.7,57.8,25.6,2.2,10.3,5.9,1.3,1.5,9.1,29.4,1.58,6.6,27.3,12.0,64.4,31.4,21.8,34.1,2.0,-9.9,9.0,0.3,-22.9,5.4,0.3,3.4,9.0,0.4,3.5,42.1,1.5,-1.8,2.0,2.1,1.6,0.01,5.5,4.1,8.1,72.6,0.4,30.7,0.3,41.8,0.4,40.0,0.2,26.8,0.6,25.7,0.155,1.7,0.429,0.076,0.5,0.622,0.050,0.3,0.720,0.256,1.4,0.640,0.120,0.9,0.558,0.114,1.1,0.471,0.018,0.2,0.593,0.080,0.4,0.737,0.023,0.3,0.371,0.064,0.5,0.678,54.3,0.6,12.2
3,Joel Embiid,PHI,F-C,27.5,48.4,30.0,2.5,11.1,3.7,0.7,1.9,5.8,18.5,1.04,7.5,29.2,11.6,59.3,32.7,19.6,49.5,4.8,-9.4,10.2,0.6,-20.6,7.5,0.7,-1.4,12.4,0.9,1.7,20.5,1.4,2.9,3.9,1.2,1.4,0.02,10.4,2.7,5.4,66.0,0.9,41.5,0.8,38.0,0.6,35.7,0.2,28.8,1.1,30.6,0.064,0.8,0.384,0.030,0.3,0.474,0.076,1.0,0.436,0.100,1.0,0.504,0.299,2.6,0.501,0.155,1.9,0.410,0.009,0.1,0.222,0.088,0.6,0.670,0.027,0.5,0.310,0.082,0.6,0.597,58.0,0.4,7.0
4,LeBron James,LAL,F,27.4,51.0,33.9,1.0,7.4,8.3,1.3,0.6,2.1,37.6,2.30,2.9,19.3,10.2,58.8,31.1,18.8,32.5,2.0,-0.2,9.9,0.4,-12.3,5.9,0.3,5.2,9.9,0.2,-17.4,41.8,1.5,-0.6,0.7,1.8,1.5,0.11,3.2,2.5,6.2,69.0,0.3,25.7,0.6,47.9,0.6,34.4,0.9,42.3,1.5,34.1,0.175,2.3,0.412,0.281,3.2,0.454,0.023,0.2,0.667,0.214,1.3,0.649,0.082,0.8,0.532,0.059,0.9,0.388,0.014,0.1,0.632,0.037,0.3,0.708,0.020,0.3,0.296,0.026,0.2,0.730,54.8,0.7,17.2


In [8]:
all = PCA(all_data)
all.PCA_variance()

,Variance Explained,Percentage of Variance Explained,Cumulative Percentage of Variance Explained
PC1,20.799200,25.0,25.0
PC2,15.397525,19.0,44.0
PC3,5.033836,6.0,50.0
PC4,2.965264,4.0,54.0
PC5,2.948295,4.0,58.0
PC6,2.651697,3.0,61.0
PC7,2.130522,3.0,64.0
PC8,1.887024,2.0,66.0
PC9,1.551563,2.0,68.0
PC10,1.513770,2.0,70.0


In [9]:
all_df = all.PCA_transform()

In [10]:
all.PCA_sorted_eigen('PC1')[:20]

OREB%                  0.197592
SCREEN ASSISTS         0.191080
3PT_FREQ              -0.186602
PNR_ROLL_FREQ          0.186350
<6_FREQ                0.186266
OREB                   0.177581
CUT_FREQ               0.173639
PNR_ROLL_FG%           0.173373
CONTESTED 2PT SHOTS    0.170080
DREB%                  0.167850
PNR_ROLL_FGM           0.165518
CUT_FGM                0.161946
PNR_HANDLER_FG%       -0.156130
PNR_HANDLER_FREQ      -0.154839
FG%                    0.152740
<6_DFGM                0.148458
<6_DIFF%              -0.147174
BLK                    0.145913
HANDOFF_FG%           -0.141014
AST TO PASS% ADJ      -0.139719
Name: PC1, dtype: float64

In [11]:
all.PCA_transform().nlargest(10, 'PC1')

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PLAYER,TEAM,POSITION
67,14.249606,5.000594,-1.074717,-3.657935,1.777759,3.423773,0.221198,-0.332974,-0.896239,-0.469011,-0.328075,0.407395,1.265338,0.436488,-0.448224,Rudy Gobert,UTA,C
58,13.072730,4.514734,-0.939064,-4.610623,0.355975,3.479539,3.122266,-0.097630,-1.265548,-0.721360,0.518667,-0.629917,1.366111,0.271868,-0.026548,Clint Capela,HOU,C
50,12.505245,6.229176,-0.606841,-1.786734,-1.899912,1.771759,3.165849,-2.338722,-1.703849,-0.146515,-0.413569,1.022965,-1.394435,0.067119,2.091260,Andre Drummond,DET,C
145,11.954843,1.252299,-1.169913,-2.371905,1.932416,4.054224,1.729745,-1.379736,0.044559,-0.308363,-0.095385,0.776874,2.283260,-0.181196,-0.475450,Jarrett Allen,BKN,C
91,11.595691,4.742111,-1.723362,-2.532327,0.186205,3.118970,3.114465,-1.972641,-0.475499,1.280358,1.302090,-1.416265,-1.387835,-1.052500,-0.727783,Steven Adams,OKC,C
62,11.288797,6.062883,-0.892372,0.583854,0.989179,0.442747,2.048594,-3.717971,0.286857,0.546533,-0.002415,-1.452932,-0.917206,-0.267739,0.479573,Deandre Ayton,PHX,C
197,11.094395,-0.727420,-1.943842,-0.775974,1.925809,-1.004726,3.630717,-0.174502,-1.553249,-0.721752,0.979155,0.760631,0.339045,-0.247887,-1.405127,Ivica Zubac,LAC,C
245,10.846870,0.022405,-0.596155,-4.414240,1.347923,-1.706901,3.884036,-0.971247,2.144384,1.281582,4.405442,1.524652,-0.780052,-0.673698,-2.795929,Mitchell Robinson,NYK,C
72,10.783133,4.485939,0.111308,4.108985,1.803163,-0.284130,-0.527754,-2.866668,-1.200548,-0.609898,-0.377185,2.984061,-1.215300,-0.774167,2.069110,Jonas Valanciunas,MEM,C
141,10.742661,0.942956,-3.796785,-1.995820,1.204606,1.584287,0.590078,3.620966,-0.326698,-0.755805,-2.321924,-2.433990,1.100329,0.847167,1.829722,DeAndre Jordan,NYK,C


In [12]:
all.PCA_sorted_eigen('PC2')[:20]

PTS                      0.238898
PIE                      0.203314
LOOSE BALLS RECOVERED    0.202976
PASSES MADE              0.201246
USG%                     0.200529
FGM_<5                   0.193003
AST                      0.182397
FGM_5-9                  0.177011
DREB                     0.173831
SECONDARY AST            0.173082
FGM_10-14                0.173018
STL                      0.166508
3PT_DFGM                 0.166003
ISOLATION_FGM            0.162692
DEFLECTIONS              0.159699
CONTESTED 3PT SHOTS      0.157451
FGM_15-19                0.153725
TRANSITION_FGM           0.150434
ISOLATION_FREQ           0.147071
ISOLATION_FG%            0.142677
Name: PC2, dtype: float64

In [13]:
all.PCA_transform().nlargest(10, 'PC2')

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PLAYER,TEAM,POSITION
0,-4.031557,13.412326,-3.632878,1.631335,-1.885437,-3.810644,0.884773,4.535974,-2.323009,-0.054895,-0.713712,6.834321,4.303252,0.439010,-2.647870,James Harden,HOU,G
3,7.744786,10.630323,1.505087,2.989003,-0.376304,1.170620,-0.166749,-0.996413,-2.906023,2.074358,-1.913973,-0.510724,-0.223964,0.847863,1.867223,Joel Embiid,PHI,F-C
2,3.229130,10.224854,-0.194595,-2.957345,-1.885142,-3.187067,-0.078454,1.929815,-2.387792,0.821875,-0.745582,1.563327,-0.605723,1.600043,2.291073,Giannis Antetokounmpo,MIL,F
9,5.631086,10.107490,1.644436,1.794815,-1.171594,0.108933,-1.432490,1.731264,-1.371201,1.692223,-0.330607,-1.701777,-2.622222,-0.332425,0.978448,Anthony Davis,NOP,F-C
18,-3.571850,9.777059,-3.643756,0.134584,-3.975623,-1.460354,0.479543,3.260109,-2.467861,0.362321,-1.272630,0.180917,-1.443251,1.466462,0.291735,Russell Westbrook,OKC,G
31,6.013722,9.523184,-0.498005,2.847592,-0.566490,0.748257,-1.233591,-1.404317,-0.347503,-2.321906,0.640530,0.024390,-2.666936,0.166613,1.460818,Nikola Jokic,DEN,C
1,-4.056028,9.452138,2.918992,-0.759261,-1.869097,-0.458240,1.584018,3.651629,1.036454,1.189653,-0.136208,1.262547,-0.753414,-0.250929,0.247988,Paul George,OKC,F
8,-2.385947,9.291106,0.240150,1.192543,2.148632,-3.636059,-1.436627,2.843610,1.200546,3.174026,-0.172927,-0.366454,-1.184665,0.884989,-0.695535,Kevin Durant,GSW,F
4,-1.401341,9.265636,-1.950025,-0.275184,-1.594073,-3.498045,0.172947,2.627830,-2.884230,-1.263321,0.723228,1.712936,0.046473,1.165521,0.028615,LeBron James,LAL,F
14,7.945427,9.264323,2.229708,4.200725,-0.973295,0.878751,-0.995777,-1.743911,-1.340924,0.096192,-0.390609,0.580999,0.713778,0.456102,1.903777,Karl-Anthony Towns,MIN,C


In [14]:
all.PCA_sorted_eigen('PC3')[:20]

SPOT_UP_FGM            0.285721
AST%                  -0.278633
SPOT_UP_FREQ           0.247577
OFF_SCREEN_FREQ        0.234665
FGM_20-24              0.230972
TO RATIO              -0.212883
OFF_SCREEN_FGM         0.211960
PNR_HANDLER_FREQ      -0.204740
3P%                    0.191448
AST                   -0.191312
OFF_SCREEN_FG%         0.187662
AST TO PASS% ADJ      -0.178415
AST/TO                -0.169282
FG%_25-29              0.163371
FGM_25-29              0.155894
PNR_HANDLER_FGM       -0.144610
CONTESTED 3PT SHOTS    0.137685
SPOT_UP_FG%            0.133329
FG%_20-24              0.126420
CUT_FG%                0.122405
Name: PC3, dtype: float64

In [15]:
all.PCA_transform().nlargest(10, 'PC3')

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PLAYER,TEAM,POSITION
19,-3.689545,5.179207,6.002304,0.935984,4.799969,-0.358352,1.737442,3.707115,1.146251,0.020418,-0.430986,-0.284267,-2.861729,0.951496,-0.294706,Klay Thompson,GSW,G
115,3.798212,3.148725,5.123692,0.348581,0.691293,4.005347,-2.543921,-1.715045,-0.300522,2.531423,-1.283129,1.569825,1.892207,-1.560302,-2.825953,Brook Lopez,MIL,C
95,-1.978171,0.855649,4.601046,-0.590740,2.867014,2.018935,0.465983,0.124546,-0.001087,-1.428033,0.996159,0.436826,1.400147,0.970080,2.054224,Joe Harris,BKN,F-G
163,-2.493771,1.331342,4.470949,-1.723456,0.045200,2.398622,-2.508114,0.212452,0.223608,1.233000,0.202497,0.045527,0.321161,0.959559,-0.792076,Danny Green,TOR,G-F
189,-2.109415,-2.507707,4.360297,0.116596,0.841336,-0.799076,-0.382271,0.650364,-0.222022,-0.156180,-1.001784,0.305470,0.531200,-1.042027,0.403506,Gerald Green,HOU,G-F
166,1.740397,-0.439794,4.292761,1.323478,-2.644630,1.181203,-0.518312,0.058556,0.857783,0.046452,1.433064,-1.263743,-0.543158,-0.456515,-0.818303,Marvin Williams,CHA,F
30,-3.926138,4.492434,4.226806,-0.948355,0.972569,0.304141,1.909940,0.403410,0.697019,0.589154,-0.538438,0.993617,1.168533,0.387811,0.226548,Buddy Hield,SAC,G
222,-1.050781,-1.922122,4.195854,-0.454265,0.045284,0.499811,-2.809451,0.347674,0.333052,-0.407424,0.664197,0.578701,0.203969,0.192870,-0.463300,Davis Bertans,SAS,F
46,-5.613708,1.362174,4.160823,0.133737,6.577399,1.951949,1.733008,2.847152,-0.625243,-1.621386,0.678249,-0.196549,-0.508616,-1.882768,0.588653,JJ Redick,PHI,G
38,2.464804,3.979749,4.071610,3.078242,-0.556806,0.042249,1.212162,-1.317974,-0.303960,-0.840625,-1.011705,0.766638,0.462137,-0.676853,0.579728,Lauri Markkanen,CHI,F


In [16]:
all.PCA_sorted_eigen('PC4')[:20]

TRANSITION_FREQ   -0.272606
TRANSITION_FG%    -0.253737
CUT_FG%           -0.232104
POST_UP_FGM        0.224053
POST_UP_FREQ       0.223219
CUT_FREQ          -0.219014
FG%               -0.195815
USG%               0.182878
FGM_5-9            0.181884
DEFLECTIONS       -0.179179
PNR_ROLL_FGM       0.177464
TS%               -0.173688
STL               -0.170914
FGM_10-14          0.159388
FG%_25-29          0.147756
FGM_15-19          0.144384
FG%_<5            -0.139528
3P%                0.137973
PUTBACK_FREQ      -0.136915
+/-               -0.134450
Name: PC4, dtype: float64

In [17]:
all.PCA_transform().nlargest(10, 'PC4')

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PLAYER,TEAM,POSITION
21,8.928666,7.842155,0.617570,5.703944,2.832712,0.520705,-1.129628,-3.453841,-1.912092,3.814333,-0.766965,-1.809682,-2.322395,0.747507,0.352066,LaMarcus Aldridge,SAS,C-F
249,0.771513,-3.038027,2.048811,5.395279,0.251039,-0.114857,-0.420876,-0.128266,-0.212819,2.489137,-1.114966,0.615836,-1.577882,0.365337,-1.834961,Dirk Nowitzki,DAL,F
28,9.119869,8.719130,1.226681,5.184279,1.094222,1.563186,-1.901111,-2.515665,-2.211316,-0.118082,0.128620,-0.342308,-1.230013,-0.010813,1.059554,Nikola Vucevic,ORL,C
369,3.069267,-4.549796,-3.345134,4.724029,1.254274,2.934658,-1.419469,1.032014,0.647126,-0.466550,0.706624,3.813885,-0.221037,-0.497579,1.890929,Pau Gasol,MIL,C-F
103,1.266052,-0.233952,2.884522,4.570133,-3.342190,-2.212370,1.160663,-0.018502,3.874243,0.757220,-1.581736,0.752818,1.168673,0.125521,-2.885555,Carmelo Anthony,HOU,F
14,7.945427,9.264323,2.229708,4.200725,-0.973295,0.878751,-0.995777,-1.743911,-1.340924,0.096192,-0.390609,0.580999,0.713778,0.456102,1.903777,Karl-Anthony Towns,MIN,C
72,10.783133,4.485939,0.111308,4.108985,1.803163,-0.284130,-0.527754,-2.866668,-1.200548,-0.609898,-0.377185,2.984061,-1.215300,-0.774167,2.069110,Jonas Valanciunas,MEM,C
101,5.168916,3.285936,-0.605459,4.056849,-1.189041,1.861020,-2.989252,2.293720,-0.826278,-0.632839,0.583423,-0.167568,-2.016358,-1.157851,-0.211626,Marc Gasol,TOR,C
300,0.844770,-4.322474,1.334100,3.903460,-1.309536,-0.021922,-1.454420,-0.252915,-1.570063,-0.734591,0.189496,-0.006672,1.027052,0.709524,-0.781149,Henry Ellenson,NYK,F
179,7.667067,-0.180253,-1.017920,3.876127,2.311979,0.945848,-1.159177,-1.244330,1.544160,1.886896,-0.144986,-1.443493,-0.600075,-0.522778,0.102982,Robin Lopez,CHI,C


In [18]:
all.PCA_sorted_eigen('PC5')[:20]

HANDOFF_FREQ             0.255979
OFF_SCREEN_FREQ          0.252650
DEFLECTIONS             -0.235774
16-3PT_FREQ              0.234116
OFF_SCREEN_FGM           0.231784
STL                     -0.222726
HANDOFF_FGM              0.222394
10-15_FREQ               0.190832
CONTESTED 3PT SHOTS     -0.181680
3PT_DFGM                -0.178923
TS%                      0.177172
SPOT_UP_FREQ            -0.173969
SPOT_UP_FGM             -0.159827
FG%_5-9                  0.159361
FG%                      0.159255
OFF_SCREEN_FG%           0.150500
6-9_FREQ                 0.149654
FGM_15-19                0.139389
LOOSE BALLS RECOVERED   -0.135797
TRANSITION_FREQ         -0.133964
Name: PC5, dtype: float64

In [19]:
all.PCA_transform().nlargest(10, 'PC5')

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PLAYER,TEAM,POSITION
46,-5.613708,1.362174,4.160823,0.133737,6.577399,1.951949,1.733008,2.847152,-0.625243,-1.621386,0.678249,-0.196549,-0.508616,-1.882768,0.588653,JJ Redick,PHI,G
27,-4.762620,5.986515,0.779875,0.274153,5.157132,1.897917,0.538619,-0.693190,-1.424215,1.496495,-0.208853,0.453159,0.771532,-1.366688,-1.396738,CJ McCollum,POR,G
19,-3.689545,5.179207,6.002304,0.935984,4.799969,-0.358352,1.737442,3.707115,1.146251,0.020418,-0.430986,-0.284267,-2.861729,0.951496,-0.294706,Klay Thompson,GSW,G
325,7.518419,-1.999598,-2.743127,-3.912738,4.252019,1.202575,-1.099605,1.373049,5.850054,1.301599,-0.730236,-1.781181,1.860264,-2.102063,0.684003,Damian Jones,GSW,C
192,-3.931458,-2.056729,3.251410,0.692700,4.014044,2.713213,-0.201068,0.266358,0.079808,-1.364585,0.659869,0.468128,-0.310783,0.282650,0.262999,Landry Shamet,LAC,G
162,-4.489822,-1.434867,3.178308,-0.324171,3.955792,1.496062,0.824002,1.311059,-0.652632,0.218017,0.662792,-0.363780,0.298056,-2.907014,1.598516,Wayne Ellington,DET,G
248,-1.391116,-3.707510,3.339102,-0.787415,3.934840,-1.470100,-0.288997,-0.385905,1.149382,-1.512482,1.459306,1.624490,-0.752072,0.631534,1.120317,Doug McDermott,IND,F
206,-3.973441,-2.444544,3.824363,0.536116,3.909735,0.758872,0.715342,1.888605,0.516210,0.898530,0.367184,0.205654,0.435617,0.046180,2.156276,Kyle Korver,UTA,G-F
79,-5.337619,2.038650,3.632340,0.358058,3.672232,0.197397,1.834620,1.603250,1.826373,0.746186,0.749987,-0.385021,-0.323784,0.065036,0.794117,Terrence Ross,ORL,G-F
224,-4.816555,-2.199053,0.520586,0.142971,3.536205,0.158427,-0.959451,-0.660797,-0.039878,-0.096194,-0.143575,0.466724,0.321435,-0.389941,1.104622,Seth Curry,POR,G


In [20]:
all.PCA_sorted_eigen('PC6')[:20]

16-3PT_DFGM            0.357542
10-15_DFGM             0.271653
16-3PT_FREQ            0.244876
6-9_DFGM               0.241628
TRANSITION_FG%        -0.229633
ISOLATION_FREQ        -0.215937
SPOT_UP_FG%           -0.191844
10-15_FREQ             0.183958
ISOLATION_FGM         -0.169040
CONTESTED 2PT SHOTS    0.156854
POST_UP_FG%           -0.149580
USG%                  -0.145842
CHARGES DRAWN          0.144012
AST/TO                 0.133219
16-3PT_DIFF%           0.132129
ISOLATION_FG%         -0.130721
6-9_FREQ               0.130193
FG%                   -0.127750
FGM_10-14             -0.127671
PASSES MADE            0.121740
Name: PC6, dtype: float64

In [21]:
all.PCA_transform().nlargest(10, 'PC6')

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PLAYER,TEAM,POSITION
387,3.139209,-7.205312,-3.718400,1.996453,0.279494,5.253720,1.649536,3.320739,1.629704,2.546081,-4.114741,3.608001,-3.029857,1.551371,-1.136294,Quincy Acy,PHX,F
234,-0.324836,-3.564711,-2.367096,1.914977,0.231854,4.364747,0.920053,0.349570,2.010694,3.062372,0.464597,0.712276,1.181502,1.691323,1.639296,Rodney McGruder,LAC,F-G
111,1.877886,0.888777,-1.867852,0.664106,-3.404786,4.144920,1.550398,1.639990,3.042091,0.002307,3.703054,0.786437,-0.325329,-0.248067,1.948216,Thaddeus Young,IND,F
145,11.954843,1.252299,-1.169913,-2.371905,1.932416,4.054224,1.729745,-1.379736,0.044559,-0.308363,-0.095385,0.776874,2.283260,-0.181196,-0.475450,Jarrett Allen,BKN,C
110,-4.362810,3.846431,-2.780877,-0.976314,-0.146467,4.013187,-1.004803,-0.041391,-0.412610,-0.739384,-0.599461,-0.487549,0.169823,-2.149456,-0.007295,Ricky Rubio,UTA,G
115,3.798212,3.148725,5.123692,0.348581,0.691293,4.005347,-2.543921,-1.715045,-0.300522,2.531423,-1.283129,1.569825,1.892207,-1.560302,-2.825953,Brook Lopez,MIL,C
106,-0.578133,-1.392941,-1.851596,2.645079,-1.719034,3.991069,2.609666,0.586101,2.364157,1.293306,2.476206,-0.422692,1.604754,0.812196,1.219540,Cedi Osman,CLE,F
382,-4.750130,-7.756118,0.586817,3.518196,-0.977927,3.896986,5.189358,-0.939451,-1.539825,-1.614165,1.256632,-3.952806,-1.517663,6.617152,-2.878487,Dairis Bertans,NOP,G
236,-1.758441,-2.640567,-3.935762,2.579346,-1.185277,3.507502,1.394607,0.639677,0.839074,0.896811,3.146714,-0.553877,1.168925,0.788937,0.672531,Shelvin Mack,CHA,G
58,13.072730,4.514734,-0.939064,-4.610623,0.355975,3.479539,3.122266,-0.097630,-1.265548,-0.721360,0.518667,-0.629917,1.366111,0.271868,-0.026548,Clint Capela,HOU,C


In [115]:
all_df = all_df[['PLAYER', 'TEAM', 'POSITION', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6']]
all_df.head()


,PLAYER,TEAM,POSITION,PC1,PC2,PC3,PC4,PC5,PC6
0,James Harden,HOU,G,-4.031557,13.412326,-3.632878,1.631335,-1.885437,-3.810644
1,Paul George,OKC,F,-4.056028,9.452138,2.918992,-0.759261,-1.869097,-0.458240
2,Giannis Antetokounmpo,MIL,F,3.229130,10.224854,-0.194595,-2.957345,-1.885142,-3.187067
3,Joel Embiid,PHI,F-C,7.744786,10.630323,1.505087,2.989003,-0.376304,1.170620
4,LeBron James,LAL,F,-1.401341,9.265636,-1.950025,-0.275184,-1.594073,-3.498045


In [116]:
all_df.to_csv('PCA_data.csv', index = False)

In [117]:
lol stop here

SyntaxError: invalid syntax (<ipython-input-117-7e9685a63a41>, line 1)